<a href="https://colab.research.google.com/github/MarcosNetoCabrera/Impact-Lab/blob/main/C%C3%B3pia_de_INDECS_AUTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from  matplotlib import pyplot as plt
from google.colab import drive
from google.colab.patches import cv2_imshow
from tensorflow.keras.models import Sequential, model_from_json, Model
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Conv2D, LeakyReLU, Reshape, BatchNormalization, Flatten, Dense, Dropout, AveragePooling2D, MaxPool2D, GlobalAveragePooling2D, Input,  MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet50, InceptionV3, EfficientNetB1, EfficientNetB1,EfficientNetB2, VGG19, EfficientNetB5
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import load_model
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -u '/content/drive/MyDrive/INDECS.zip' -d '/content/Images'
base_dir = '/content/INDECS'

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: /content/Images/INDECS/Sunny_dumbo_12/t1139101103.427185_rEO_x4.38278_y16.6832_a2.53752.png  
  inflating: /content/Images/INDECS/Sunny_dumbo_12/t1139101103.626740_rEO_x4.29673_y16.7571_a2.50597.png  
  inflating: /content/Images/INDECS/Sunny_dumbo_12/t1139101103.826804_rEO_x4.22692_y16.8125_a2.49237.png  
  inflating: /content/Images/INDECS/Sunny_dumbo_12/t1139101104.026726_rEO_x4.14489_y16.8647_a2.53048.png  
  inflating: /content/Images/INDECS/Sunny_dumbo_12/t1139101104.226719_rEO_x4.05671_y16.9081_a2.57372.png  
  inflating: /content/Images/INDECS/Sunny_dumbo_12/t1139101104.426720_rEO_x3.96022_y16.9622_a2.59951.png  
  inflating: /content/Images/INDECS/Sunny_dumbo_12/t1139101104.626727_rEO_x3.88391_y17.0121_a2.59137.png  
  inflating: /content/Images/INDECS/Sunny_dumbo_12/t1139101104.826714_rEO_x3.80063_y17.0632_a2.58535.png  
  inflating: /content/Images/INDECS/Sunny_dumbo_12/t1139101105.026833_rEO_x3.80063_y1

In [ ]:
input_shape=(112,112,3)

In [ ]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (112, 112))
    # img = img / 255.0
    img = np.array(img)
    img = np.expand_dims(img, axis=0)

    return img

In [ ]:
def get_image_label(filename, labels):
    for label in labels:
        if filename.find(label) != -1:
            if(label == 'rPR' or label == 'rPA'):
              return 0
            elif(label == 'rEO'):
              return 1
            elif(label == 'rKT'):
              return 2
            elif(label == 'rCR'):
              return 3
            else:
              return 4

In [ ]:
def load_data(directory, labels):
    images = []
    image_paths = []
    image_labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_paths.append(os.path.join(directory, filename))
            label = get_image_label(filename, labels)
            image_labels.append(label)

    for image_path in image_paths:
        image = preprocess_image(image_path)
        images.append(image)

    images = np.vstack(images)
    test_labels = np.array(image_labels)

    # one hot encode target values
    image_labels = to_categorical(image_labels,num_classes=5)
    print(image_labels.shape)
    # print(labels[5])

    # convert from integers to floats
    images = images.astype('float32')
    print(images.shape)

    return images, image_labels, test_labels

In [ ]:
labels_1 = ['rPR', 'rEO', 'rKT', 'rCR', 'rBO']
labels_2 = ['rPA', 'rEO', 'rKT', 'rCR', 'rBO']

In [ ]:
base_cloudy = ("/content/Images/INDECS/Cloudy")
base_sunny = ("/content/Images/INDECS/Sunny")
base_night = ("/content/Images/INDECS/Night")

In [ ]:
X_train, y_train, test_labels_train = load_data(base_cloudy,labels_1)

(1080, 5)
(1080, 112, 112, 3)


In [ ]:
X_test_sunny, y_test_sunny, test_labels_sunny  = load_data(base_sunny,labels_1)
X_test_night, y_test_night, test_labels_night = load_data(base_night,labels_1)

(1080, 5)
(1080, 112, 112, 3)
(1092, 5)
(1092, 112, 112, 3)


In [ ]:
print(labels_1)

['rPR', 'rEO', 'rKT', 'rCR', 'rBO']


In [ ]:
# cv2_imshow(X_train[250])
# print(y_train[250])

In [ ]:
# cv2_imshow(X_test_night[45])
# print(y_test_night[45])

In [ ]:
# Normaliza apenas os dados para o autoencoder
X_train_autoencoder = X_train / 255.0
X_sunny_auto = X_test_sunny / 255.0
X_night_auto = X_test_night / 255.0

In [ ]:
# # Encoder
# input_img = Input(shape=input_shape)
# x = Conv2D(64, (3, 3), padding='same')(input_img)
# x = BatchNormalization()(x)
# x = LeakyReLU(alpha=0.3)(x)
# x = MaxPooling2D((2, 2), padding='same')(x)

# x = Conv2D(64, (3, 3), padding='same')(x)
# x = BatchNormalization()(x)
# x = LeakyReLU(alpha=0.3)(x)
# x = MaxPooling2D((2, 2), padding='same')(x)

# encoded = Conv2D(32, (3, 3), padding='same')(x)
# encoded = BatchNormalization()(encoded)
# encoded = LeakyReLU(alpha=0.2)(encoded)

# # Decoder
# x = Conv2D(32, (3, 3), padding='same')(encoded)
# x = BatchNormalization()(x)
# x = LeakyReLU(alpha=0.3)(x)
# x = UpSampling2D((2, 2))(x)

# x = Conv2D(16, (3, 3), padding='same')(x)
# x = BatchNormalization()(x)
# x = LeakyReLU(alpha=0.3)(x)
# x = UpSampling2D((2, 2))(x)

# decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

In [ ]:
# # Criar o modelo do autoencoder
# autoencoder = Model(input_img, decoded)
# autoencoder.compile(optimizer='adam', loss='mse')

In [ ]:
# # Treinar o autoencoder
# autoencoder.fit(X_train_autoencoder, X_train_autoencoder, batch_size=16, epochs=50, validation_split=0.2)

In [ ]:
!pip install scikit-learn keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
import keras_tuner

def build_autoencoder(hp):

    alpha = hp.Choice('alpha', [0.2, 0.25 ,0.3, 0.35])

    input_img = Input(shape=input_shape)
    x = Conv2D(hp.Choice('filters1', [256, 128, 64, 32]), (3, 3), padding='same')(input_img)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(hp.Choice('filters2', [256, 128, 64, 32]), (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

    encoded = Conv2D(hp.Choice('filters3', [256, 128, 64, 32]), (3, 3), padding='same')(x)
    encoded = BatchNormalization()(encoded)
    encoded = LeakyReLU(alpha=alpha)(encoded)

    # Decoder
    x = Conv2D(hp.Choice('decoder1', [32]), (3, 3), padding='same')(encoded)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = UpSampling2D((2, 2))(x)

    x = Conv2D(hp.Choice('decoder2',[16]), (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = UpSampling2D((2, 2))(x)

    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')

    return autoencoder

In [ ]:
tuner = keras_tuner.GridSearch(
build_autoencoder,
objective='val_loss',
directory='grid_search_results',
project_name='autoencoder_grid_search',
)

In [ ]:
tuner.search(X_train_autoencoder, X_train_autoencoder, batch_size=16, epochs=30, validation_split=0.2)

Trial 142 Complete [00h 01m 25s]
val_loss: 0.002570971380919218

Best val_loss So Far: 0.0017862378153949976
Total elapsed time: 03h 19m 50s

Search: Running Trial #143

Value             |Best Value So Far |Hyperparameter
0.3               |0.25              |alpha
256               |32                |filters1
32                |256               |filters2
64                |256               |filters3
32                |32                |decoder1
16                |16                |decoder2

Epoch 1/30


In [ ]:
best_model = tuner.get_best_hyperparameters()[0]
print(best_model.values)

# best_model = tuner.get_best_models()[0]
# best_model.summary()

In [ ]:
# Extração de recursos com o autoencoder
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('leaky_re_lu_2').output)
X_train_encoder = encoder.predict(X_train_autoencoder)
X_sunny_encoder = encoder.predict(X_sunny_auto)
X_night_encoder = encoder.predict(X_night_auto)

In [ ]:
conv_base = EfficientNetB2(include_top=False,weights='imagenet',input_shape=input_shape)

In [ ]:
for layer in conv_base.layers:
    layer.trainable = False

In [ ]:
# Adicionar as camadas do autoencoder ao modelo pré-treinado
encoded_input = Input(shape=(X_train_encoder.shape[1],))
x = conv_base.layers[-1].output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(5, activation='softmax')(x)

In [ ]:
opt = Adam(learning_rate=0.0001)
model = Model(inputs=[conv_base.input, encoded_input], outputs=x)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Treinar o modelo final
history = model.fit([X_train, X_train_encoder], y_train, batch_size=16, epochs=10, validation_data=([X_test_sunny, X_sunny_encoder], y_test_sunny), verbose=1)

In [ ]:
results_sunny = model.evaluate([X_test_sunny, X_sunny_encoder], y_test_sunny)

In [ ]:
results_night = model.evaluate([X_test_night, X_night_encoder], y_test_night)

In [ ]:
pC = model.predict([X_test_sunny,X_sunny_encoder])
predC = np.argmax(pC, axis = 1)
pS = model.predict([X_test_night,X_night_encoder])
predS = np.argmax(pS, axis = 1)

In [ ]:
print(classification_report(predC,test_labels_sunny))

In [ ]:
print(classification_report(predS,test_labels_night))

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)

    # Show all ticks and label them with the respective list entries
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
plot_confusion_matrix(test_labels_sunny, predC, classes=labels_1, normalize=False, title='Confusion Matrix Sunny')
plt.show()

In [ ]:
plot_confusion_matrix(test_labels_night, predS, classes=labels_1, normalize=False, title='Confusion Matrix Night')
plt.show()